# Inertio-gravity response to mass sink events: a demo 
### (estimated from cold-IR blobs) 

##### Calculations for plot_expanding_ring_coriolis:

The shading of annuli of subsidence or warmth is arbitrary in absolute value. It obeys the principle that ink is conserved as rings expand, until they are removed from the buffer when their opacity alpha drops below 0.01 for computational economy. But the absolute values of alpha have no other meaning. 

For Coriolis pinwheels, a more quanitative model of convection is needed. Here I suppose `size` of the event (cold cloud) in square degrees is equal to the area that was removed by convergence in that circle. As if a disc of low-level air rose up to the tropopause and became the observed area of each cold cloud event. If a ring of parcels contracts by a radial distance dr, the area of the contraction annulus is $2 \pi r dr$, which we set equal to `size`. Tangential velocity is then `vtan = f*dr`, tangential distance traversed up to the time of the depiction is `dxtan = f*dr*age`, and for a small angle (where x = sinx = tanx suffices), `dtheta = dxtan / (2*pi*r)'



In [1]:
import os 
import numpy as np
import xarray as xr
from datetime import date, timedelta, datetime
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from matplotlib.image import imread
import geocat.viz.util as gvutil
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from collections import deque
import matplotlib.patches as patches
from pathlib import Path
import json

In [94]:
class EyedropperAnnotator:
    def __init__(self):
        self.brown_events = deque()
        
    def add_events(self, centroids, sizes, current_time):
        """Add new events with sizes at current time"""
        for (lat, lon), size in zip(centroids, sizes):
            self.brown_events.append((lon, lat, current_time, size))
    
    def save_state(self, filepath):
        """Overwrite current state (for restart)"""
        def convert_event(event):
            return tuple(float(x) if isinstance(x, (np.floating, np.integer)) else x for x in event)
        
        state = {'brown_events': [convert_event(e) for e in self.brown_events]}
        with open(filepath, 'w') as f:
            json.dump(state, f)

    def append_history(self, filepath):
        """Append all events to growing file"""
        def convert_event(event):
            return tuple(float(x) if isinstance(x, (np.floating, np.integer)) else x for x in event)
        
        snapshot = {'brown_events': [convert_event(e) for e in self.brown_events]}
        with open(filepath, 'a') as f:
            f.write(json.dumps(snapshot) + '\n')

    @staticmethod
    def load_state(filepath):
        """Load state for restart, or create new if file doesn't exist"""
        if not Path(filepath).exists():
            return EyedropperAnnotator()
        with open(filepath, 'r') as f:
            state = json.load(f)
        annotator = EyedropperAnnotator()
        annotator.brown_events = deque(state['brown_events'])
        return annotator


    
    def plot_one_expanding_ring(self, ax, current_time, facecolor='orange'):
        """Plot expanding annuli with size-dependent opacity"""
        import matplotlib.patches as patches
        import cartopy.crs as ccrs
        
        to_remove = []
        for i, (lon, lat, birth_time, size) in enumerate(list(self.brown_events)):
            age = current_time - birth_time
            size= size/100.
            if age < 0:
                continue
                
            outer_r = age * 52 * 60 * 30 / 111111.1
            area = np.pi * (outer_r**2)
            alpha = min(0.8, size / max(area, 1))
            
            if alpha > 0.01:
                if outer_r > 0:
                    annulus = patches.Annulus((lon, lat), outer_r, width=outer_r/2, 
                                          facecolor=facecolor, alpha=alpha, 
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(annulus)
            else:
                to_remove.append(i)
        
        for i in reversed(to_remove):
            del self.brown_events[i]

    def plot_expanding_ring_coriolis_pinwheel(self, ax, current_time, facecolor='orange', 
                                     spoke_color='white', n_spokes=12):
        """
        Plot expanding annuli with Coriolis-curved spokes.
        
        The spokes show rotation proportional to Coriolis force.        
        Args:
            ax: matplotlib axis
            current_time: current frame time
            facecolor: color of annulus
            spoke_color: color of Coriolis spokes
            n_spokes: number of spokes (default 12)
        """
        
        to_remove = []
        for i, (lon, lat, birth_time, size) in enumerate(list(self.brown_events)):
            age = current_time - birth_time  # seconds 
            if age < 0:
                continue
                
            outer_r = age    * 52 * 60 * 30 / 111111.1  # degrees
            inner_r = (age-1)* 52 * 60 * 30 / 111111.1
            width =         1* 52 * 60 * 30 / 111111.1
            area = np.pi * (outer_r**2 - inner_r**2)
            alpha = min(0.8, size / max(area, 1))      # min and max keep it in [0,0.8] 
            
            if alpha > 0.01:
                if outer_r > 0:
                    # Draw base annulus
                    annulus = patches.Annulus((lon, lat), outer_r, width=width, 
                                          facecolor=facecolor, alpha=alpha, 
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(annulus)


                    # Draw Coriolis spokes for this age 
                    lat_rad = np.radians(lat)
                    f = 2*2*np.pi/86400. *np.sin(lat_rad)  # Coriolis , f(in /s, lat)
                    
                    # Mean counterclockwise rotation increases with age for f>0
                    # If a ring of parcels contracts by a radial distance dr, 
                    # the area of the contraction annulus is set equal to size. 
                    # then vtan = f*dr, distance traversed is dxtan = f*dr*age, 
                    # dtheta = dxtan / (2*pi*r)
 
                    
                    for spoke_idx in range(n_spokes):
                        # Base azimuth for this spoke at outer_r 
                        base_azimuth = np.radians(spoke_idx * 360 / n_spokes)
                        
                        # Create curved spoke from outer_r to inner_2
                        n_points = 20
                        radii = np.linspace(outer_r, 0*inner_r+1e-3, n_points) # avoid r=0 
                        dr = size /2/np.pi/radii *111111.   # meters 
                        vtan = f*dr      # m/s, function of latitude
                        dxtan= vtan*age                     # meters 
                        dtheta = dxtan /2/np.pi/radii       # radians
                        
                        # Azimuth(radii), varies along spoke due to Coriolis
                        azimuths = base_azimuth + dtheta 
                        
                        # Convert to lon/lat offsets
                        x_offsets = radii * np.cos(azimuths)
                        y_offsets = radii * np.sin(azimuths)
                        
                        # Create curved path
                        spoke_lons = lon + x_offsets
                        spoke_lats = lat + y_offsets
                        
                        # Plot as curved line
                        ax.plot(spoke_lons, spoke_lats, color=spoke_color, \
                                transform=ccrs.PlateCarree(), \
                                alpha = min(0.9, 3 / max(area, 1)))   
                        
            else:
                to_remove.append(i)
        
        for i in reversed(to_remove):
            del self.brown_events[i]    

# okay let's make the frames of animation 

In [99]:
# Open data files 

from pathlib import Path
datafiles = '/Volumes/Samsung USB/PrePreIrma_IRfiles/'+ \
            'merg_20170813[0,1,2]*_4km-pixel.nc4'

OUTPUT = Path('/Users/bmapes/Downloads/CoriolisPinwheels/')
OUTPUT.mkdir(exist_ok=True, parents=True) 
OUTDIR = '/Volumes/Samsung USB/CoriolisPinwheels/'
OUTDIR = '/Users/bmapes/Downloads/CoriolisPinwheels/'

ds = xr.open_mfdataset(datafiles)
Tb = ds.Tb
Tb

<xarray.DataArray 'Tb' (time: 38, lat: 1100, lon: 1374)> Size: 230MB
dask.array<concatenate, shape=(38, 1100, 1374), dtype=float32, chunksize=(2, 1100, 1374), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 5kB 0.01819 0.05457 0.09094 ... 49.89 49.93 49.97
  * lat      (lat) float32 4kB -19.99 -19.96 -19.92 -19.88 ... 19.92 19.96 19.99
  * time     (time) datetime64[ns] 304B 2017-08-13T05:00:00.000013312 ... 201...
Attributes:
    units:          K
    standard_name:  brightness_temperature

In [100]:
# Initialize annotator, and loop over times 
# annotator = EyedropperAnnotator.load_state('annotator_state_syn.pkl')
annotator = EyedropperAnnotator()

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import cartopy.crs as ccrs

# Circle (convective event, in pixels) specifications
centroids = [(15, 10), (15, 25), (15, 40)]  # (lat, lon)
start_times = [0, 10, 20]  # timestep when each event starts
durations = [4, 8, 12]  # timesteps: 2hr, 4hr, 6hr
peak_sizes = [100, 400, 2000]  # pixels in 4km imagery (from sq. deg)

# Calculate relative radius for blue circle (60% area = sqrt(0.6) of radius)
blue_scale = np.sqrt(0.6)

for it in range(len(Tb.time)):
    # For plotting
    fig = plt.figure(figsize=(10, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_facecolor('black')

    # ax.set_xlim(Tb.lon.min(), Tb.lon.max())
    # ax.set_ylim(Tb.lat.min(), Tb.lat.max())
    ax.set_xlim(-20,70)
    ax.set_ylim(-30,30)
    ax.set_aspect('equal')
    plt.axis('off')    

    # Calculate active circles
    active_centroids = []
    active_sizes = []

    # Active now? Load into the annotator, and plot 
    for (lat, lon), start_time, duration, peak_size in zip(centroids, start_times, durations, peak_sizes):
    #for (lat, lon), duration, peak_size, start_time in zip(centroids, durations, peak_sizes, start_times):

        elapsed = it - start_time
        if 0 <= elapsed < duration:
            progress = elapsed / duration
            ramp = 1 - abs(2 * progress - 1)  # Triangle: 0->1->0
            radius = np.sqrt(peak_size * ramp)*0.0364      # 4km pixels 
            # radius in degrees, from peak_size in 0.364deg pixels
            print('radius '+str(radius))
            
            # Draw circles
            circle_cyan = Circle((lon, lat), radius, color='cyan', fill=True, linewidth=2, transform=ccrs.PlateCarree())
            ax.add_patch(circle_cyan)
            
            circle_blue = Circle((lon, lat), radius * blue_scale, color='blue', fill=True, linewidth=2, transform=ccrs.PlateCarree())
            ax.add_patch(circle_blue)

            print(f"it={it}, start_time={start_time}, duration={duration}, elapsed={elapsed}")

            # Store for annotator
            active_centroids.append((lat, lon))
            active_sizes.append(radius)  # square degrees from 4km pixels

    
    # Call annotator
    annotator.add_events(active_centroids, (np.array(active_sizes)).tolist(), it)
    annotator.plot_expanding_ring_coriolis_pinwheel(ax, it, facecolor='orange', 
                                       spoke_color='white', n_spokes=8)
    
    plt.savefig(OUTDIR+f'frame_{it:04d}.png', bbox_inches='tight', pad_inches=0, \
                facecolor='black', dpi=120)
    plt.close()
    
    annotator.save_state('annotator_state_syn.pkl')        # for restart if kernel drops 
    annotator.append_history('annotator_history_syn.json') # for driving midi music events perhaps 

radius 0.0
it=0, start_time=0, duration=4, elapsed=0
radius 0.25738686835190333
it=1, start_time=0, duration=4, elapsed=1
radius 0.364
it=2, start_time=0, duration=4, elapsed=2
radius 0.25738686835190333
it=3, start_time=0, duration=4, elapsed=3
radius 0.0
it=10, start_time=10, duration=8, elapsed=0
radius 0.364
it=11, start_time=10, duration=8, elapsed=1
radius 0.5147737367038067
it=12, start_time=10, duration=8, elapsed=2
radius 0.6304664939550714
it=13, start_time=10, duration=8, elapsed=3
radius 0.728
it=14, start_time=10, duration=8, elapsed=4
radius 0.6304664939550714
it=15, start_time=10, duration=8, elapsed=5
radius 0.5147737367038067
it=16, start_time=10, duration=8, elapsed=6
radius 0.364
it=17, start_time=10, duration=8, elapsed=7
radius 0.0
it=20, start_time=20, duration=12, elapsed=0
radius 0.6645700364396016
it=21, start_time=20, duration=12, elapsed=1
radius 0.9398439586796665
it=22, start_time=20, duration=12, elapsed=2
radius 1.15106906830129
it=23, start_time=20, dura